In [83]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from scipy.stats import zscore

csv_path = "data/Landslide.csv"
device = "102"
save_path = f"data/dev{device}_prepared.csv"

df = pd.read_csv(csv_path)

df = df[~df['timestamp'].str.contains(r'\(ICT\)', na=False)]
df = df.drop(columns=['bug1', 'bug2', 'bug3', 'bug4'])
df = df[df['devID'] == int(device)]
df


C:\Users\ahmad\AppData\Local\Temp\ipykernel_20240\930771119.py:12: DtypeWarning: Columns (9,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


,timestamp,devID,soil,rain,temp,humi,geo,unixt,lat,lng
1,2025-05-02T11:42:00.384119,102.0,22.200001,0.0,34.549099,61.175941,3.323395,1.746186e+09,8.637964,99.898933
3,2025-05-02T11:42:11.034615,102.0,22.200001,0.0,34.506374,61.542152,14.766250,1.746186e+09,8.637963,99.898933
5,2025-05-02T11:42:21.675224,102.0,22.200001,0.0,34.485012,61.767223,3.242806,1.746186e+09,8.637963,99.898933
7,2025-05-02T11:42:32.443581,102.0,22.200001,0.0,34.477001,62.366138,6.375742,1.746186e+09,8.637964,99.898933
9,2025-05-02T11:42:43.142656,102.0,22.200001,0.0,34.447620,60.361488,19.010500,1.746186e+09,8.637964,99.898933
...,...,...,...,...,...,...,...,...,...,...
178441,2025-06-12 11:42:48.52,102.0,52.599998,0.0,28.431374,79.423821,3.911398,1.749753e+09,8.637935,99.89892578
178443,2025-06-12 11:42:59.22,102.0,52.799999,0.0,28.393990,79.343712,2.637993,1.749753e+09,8.637933,99.89892578
178445,2025-06-12 11:43:09.94,102.0,52.599998,0.0,28.410011,79.511559,2.706004,1.749754e+09,8.637933,99.89892578
178447,2025-06-12 11:43:20.56,102.0,52.599998,0.0,28.420692,79.326546,17.579735,1.749754e+09,8.637933,99.89892578


In [84]:
df.columns = df.columns.str.strip()
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df = df.dropna(subset=['timestamp', 'geo'])  # ต้องมี timestamp และ geo

for col in ['soil', 'rain', 'temp', 'humi', 'geo', 'lat', 'lng']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

C:\Users\ahmad\AppData\Local\Temp\ipykernel_20240\2398584010.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_numeric(df[col], errors='coerce')


,timestamp,devID,soil,rain,temp,humi,geo,unixt,lat,lng
1,2025-05-02 11:42:00.384119,102.0,22.200001,0.0,34.549099,61.175941,3.323395,1.746186e+09,8.637964,99.898933
3,2025-05-02 11:42:11.034615,102.0,22.200001,0.0,34.506374,61.542152,14.766250,1.746186e+09,8.637963,99.898933
5,2025-05-02 11:42:21.675224,102.0,22.200001,0.0,34.485012,61.767223,3.242806,1.746186e+09,8.637963,99.898933
7,2025-05-02 11:42:32.443581,102.0,22.200001,0.0,34.477001,62.366138,6.375742,1.746186e+09,8.637964,99.898933
9,2025-05-02 11:42:43.142656,102.0,22.200001,0.0,34.447620,60.361488,19.010500,1.746186e+09,8.637964,99.898933
...,...,...,...,...,...,...,...,...,...,...
178227,2025-06-12 10:46:29.535004,102.0,53.000000,0.0,30.129704,70.535439,4.859087,1.749750e+09,8.637939,99.898911
178229,2025-06-12 10:46:39.122909,102.0,53.000000,0.0,30.161747,70.760506,7.824049,1.749750e+09,8.637940,99.898911
178231,2025-06-12 10:46:49.933066,102.0,53.200001,0.0,30.193790,70.909286,15.313673,1.749750e+09,8.637940,99.898918
178233,2025-06-12 10:47:00.541802,102.0,53.200001,0.0,30.215153,71.048523,4.406499,1.749750e+09,8.637940,99.898918


In [85]:
# เรียงลำดับตาม devID และ timestamp
df = df.sort_values('timestamp').reset_index(drop=True)

# ตั้ง timestamp เป็น index
df = df.set_index('timestamp')
df

,devID,soil,rain,temp,humi,geo,unixt,lat,lng
timestamp,,,,,,,,,
2025-05-02 11:42:00.384119,102.0,22.200001,0.0,34.549099,61.175941,3.323395,1.746186e+09,8.637964,99.898933
2025-05-02 11:42:11.034615,102.0,22.200001,0.0,34.506374,61.542152,14.766250,1.746186e+09,8.637963,99.898933
2025-05-02 11:42:21.675224,102.0,22.200001,0.0,34.485012,61.767223,3.242806,1.746186e+09,8.637963,99.898933
2025-05-02 11:42:32.443581,102.0,22.200001,0.0,34.477001,62.366138,6.375742,1.746186e+09,8.637964,99.898933
2025-05-02 11:42:43.142656,102.0,22.200001,0.0,34.447620,60.361488,19.010500,1.746186e+09,8.637964,99.898933
...,...,...,...,...,...,...,...,...,...
2025-06-12 10:46:29.535004,102.0,53.000000,0.0,30.129704,70.535439,4.859087,1.749750e+09,8.637939,99.898911
2025-06-12 10:46:39.122909,102.0,53.000000,0.0,30.161747,70.760506,7.824049,1.749750e+09,8.637940,99.898911
2025-06-12 10:46:49.933066,102.0,53.200001,0.0,30.193790,70.909286,15.313673,1.749750e+09,8.637940,99.898918


In [86]:
df = df[(np.abs(zscore(df[['soil', 'rain', 'temp', 'humi', 'geo']])) < 3).all(axis=1)]
df

,devID,soil,rain,temp,humi,geo,unixt,lat,lng
timestamp,,,,,,,,,
2025-05-03 01:25:50.150783,102.0,52.599998,0.2794,27.205690,80.846725,17.881044,1.746235e+09,8.637938,99.898933
2025-05-03 01:26:02.062923,102.0,53.200001,0.2794,27.141603,81.363625,11.430539,1.746235e+09,8.637937,99.898933
2025-05-03 01:26:50.017812,102.0,52.599998,1.3970,27.106890,80.779968,16.151541,1.746235e+09,8.637936,99.898933
2025-05-03 01:27:02.201784,102.0,52.799999,1.6764,27.120241,80.419472,15.776340,1.746235e+09,8.637937,99.898933
2025-05-03 01:27:13.949639,102.0,52.500000,1.9558,27.050814,80.730377,17.829277,1.746235e+09,8.637937,99.898933
...,...,...,...,...,...,...,...,...,...
2025-06-12 10:46:29.535004,102.0,53.000000,0.0000,30.129704,70.535439,4.859087,1.749750e+09,8.637939,99.898911
2025-06-12 10:46:39.122909,102.0,53.000000,0.0000,30.161747,70.760506,7.824049,1.749750e+09,8.637940,99.898911
2025-06-12 10:46:49.933066,102.0,53.200001,0.0000,30.193790,70.909286,15.313673,1.749750e+09,8.637940,99.898918


In [ ]:
# ฟังก์ชันเติม NaN หลัง rolling
def fill_na(df_group):
    return df_group.fillna(method='bfill').fillna(method='ffill')

# กำหนด windows
rain_windows = {
    'rain_1h_sum': '1H',
    'rain_3h_sum': '3H',
    'rain_6h_sum': '6H',
    'rain_12h_sum': '12H',
    'rain_1d_sum': '1D',
    'rain_7d_sum': '7D',
}
soil_mean_windows = {
    'soil_1h_mean': '1H',
    'soil_3h_mean': '3H',
    'soil_6h_mean': '6H',
    'soil_12h_mean': '12H',
    'soil_1d_mean': '1D',
}
geo_mean_windows = {
    'geo_1h_mean': '1H',
    'geo_3h_mean': '3H',
    'geo_6h_mean': '6H',
    'geo_12h_mean': '12H',
    'geo_1d_mean': '1D',
}


# สร้าง group ตาม devID สำหรับ rolling
grouped = df.groupby('devID')

# Rolling sums/means
for col_name, window in rain_windows.items():
    df[col_name] = grouped['rain'].rolling(window).sum().reset_index(level=0, drop=True)
for col_name, window in soil_mean_windows.items():
    df[col_name] = grouped['soil'].rolling(window).mean().reset_index(level=0, drop=True)
for col_name, window in geo_mean_windows.items():
    df[col_name] = grouped['geo'].rolling(window).mean().reset_index(level=0, drop=True)

df

C:\Users\ahmad\AppData\Local\Temp\ipykernel_20240\1031305995.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = grouped['rain'].rolling(window).sum().reset_index(level=0, drop=True)
C:\Users\ahmad\AppData\Local\Temp\ipykernel_20240\1031305995.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name] = grouped['rain'].rolling(window).sum().reset_index(level=0, drop=True)
C:\Users\ahmad\AppData\Local\Temp\ipykernel_20240\1031305995.py:33: SettingWithCopyWarning: 
A value is trying to 

,devID,soil,rain,temp,humi,geo,unixt,lat,lng,rain_1h_sum,...,soil_1h_mean,soil_3h_mean,soil_6h_mean,soil_12h_mean,soil_1d_mean,geo_1h_mean,geo_3h_mean,geo_6h_mean,geo_12h_mean,geo_1d_mean
timestamp,,,,,,,,,,,,,,,,,,,,,
2025-05-03 01:25:50.150783,102.0,52.599998,0.2794,27.205690,80.846725,17.881044,1.746235e+09,8.637938,99.898933,0.2794,...,52.599998,52.599998,52.599998,52.599998,52.599998,17.881044,17.881044,17.881044,17.881044,17.881044
2025-05-03 01:26:02.062923,102.0,53.200001,0.2794,27.141603,81.363625,11.430539,1.746235e+09,8.637937,99.898933,0.5588,...,52.900000,52.900000,52.900000,52.900000,52.900000,14.655792,14.655792,14.655792,14.655792,14.655792
2025-05-03 01:26:50.017812,102.0,52.599998,1.3970,27.106890,80.779968,16.151541,1.746235e+09,8.637936,99.898933,1.9558,...,52.799999,52.799999,52.799999,52.799999,52.799999,15.154375,15.154375,15.154375,15.154375,15.154375
2025-05-03 01:27:02.201784,102.0,52.799999,1.6764,27.120241,80.419472,15.776340,1.746235e+09,8.637937,99.898933,3.6322,...,52.799999,52.799999,52.799999,52.799999,52.799999,15.309866,15.309866,15.309866,15.309866,15.309866
2025-05-03 01:27:13.949639,102.0,52.500000,1.9558,27.050814,80.730377,17.829277,1.746235e+09,8.637937,99.898933,5.5880,...,52.739999,52.739999,52.739999,52.739999,52.739999,15.813748,15.813748,15.813748,15.813748,15.813748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-12 10:46:29.535004,102.0,53.000000,0.0000,30.129704,70.535439,4.859087,1.749750e+09,8.637939,99.898911,0.0000,...,53.169231,53.169231,54.429688,57.249188,58.209472,5.593953,5.593953,5.662459,5.559111,5.076258
2025-06-12 10:46:39.122909,102.0,53.000000,0.0000,30.161747,70.760506,7.824049,1.749750e+09,8.637940,99.898911,0.0000,...,53.167089,53.167089,54.403126,57.246425,58.208539,5.622182,5.622182,5.614164,5.561876,5.076750
2025-06-12 10:46:49.933066,102.0,53.200001,0.0000,30.193790,70.909286,15.313673,1.749750e+09,8.637940,99.898918,0.0000,...,53.167501,53.167501,54.378126,57.244234,58.207465,5.743325,5.743325,5.711450,5.567155,5.078846


In [88]:

# ความต่างและ STD
df['soil_diff'] = grouped['soil'].diff().reset_index(level=0, drop=True)
df['geo_diff'] = grouped['geo'].diff().reset_index(level=0, drop=True)
df['soil_1h_std'] = grouped['soil'].rolling('1H').std().reset_index(level=0, drop=True)
df['geo_1h_std'] = grouped['geo'].rolling('1H').std().reset_index(level=0, drop=True)

# เติม NaN ที่เกิดจาก rolling
df = grouped.apply(fill_na)
df.index = df.index.droplevel(0)
df = df.reset_index()  # เอา timestamp กลับมาเป็น column


C:\Users\ahmad\AppData\Local\Temp\ipykernel_20240\3765377015.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['soil_diff'] = grouped['soil'].diff().reset_index(level=0, drop=True)
C:\Users\ahmad\AppData\Local\Temp\ipykernel_20240\3765377015.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['geo_diff'] = grouped['geo'].diff().reset_index(level=0, drop=True)
C:\Users\ahmad\AppData\Local\Temp\ipykernel_20240\3765377015.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84469 entries, 0 to 84468
Data columns (total 30 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   timestamp      84469 non-null  datetime64[ns]
 1   devID          84469 non-null  float64       
 2   soil           84469 non-null  float64       
 3   rain           84469 non-null  float64       
 4   temp           84469 non-null  float64       
 5   humi           84469 non-null  float64       
 6   geo            84469 non-null  float64       
 7   unixt          84469 non-null  float64       
 8   lat            84469 non-null  float64       
 9   lng            84469 non-null  float64       
 10  rain_1h_sum    84469 non-null  float64       
 11  rain_3h_sum    84469 non-null  float64       
 12  rain_6h_sum    84469 non-null  float64       
 13  rain_12h_sum   84469 non-null  float64       
 14  rain_1d_sum    84469 non-null  float64       
 15  rain_7d_sum    8446

In [90]:
nan_counts = df.isna().sum()

nan_columns = nan_counts[nan_counts > 0]

print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
soil_diff    84469
geo_diff     84469
dtype: int64


In [91]:
df['soil_diff'] = df['soil'].diff()
df['geo_diff'] = df['geo'].diff()

In [92]:
nan_counts = df.isna().sum()
nan_columns = nan_counts[nan_counts > 0]
print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

📌 คอลัมน์ที่มีค่า NaN:
soil_diff    1
geo_diff     1
dtype: int64


In [93]:
df = df.dropna()

nan_counts = df.isna().sum()
nan_columns = nan_counts[nan_counts > 0]
print("df102")
print("📌 คอลัมน์ที่มีค่า NaN:")
print(nan_columns)

df102
📌 คอลัมน์ที่มีค่า NaN:
Series([], dtype: int64)


In [94]:

df.to_csv(f"{save_path}", index=False)

print("✅ บันทึก dev101_prepared.csv และ dev102_prepared.csv เรียบร้อยแล้ว")

✅ บันทึก dev101_prepared.csv และ dev102_prepared.csv เรียบร้อยแล้ว
